In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
class MyNewModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(10, 20)
        self.act1 = nn.LeakyReLU(1/128)
        self.lin2 = nn.Linear(20, 40)
        self.act2 = nn.LeakyReLU(1/128)
        self.lin3 = nn.Linear(40, 3)

    def forward(self, x):
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        return self.lin3(x)


In [3]:
my_torch_model = MyNewModel()

In [4]:
indata = torch.randn(1, 10)
outdata = my_torch_model(indata)

In [7]:
# my_onnx_model = torch.onnx.dynamo_export(my_torch_model, (torch.randn(10, 1)))
torch.onnx.export(my_torch_model, (torch.randn(1, 10)),"data/demo_model.onnx")

In [16]:
import onnx
from hls.compiler import parsemodel, svimpl
def make_testing_tl(dim, tlname):
    mdl = onnx.load("data/demo_model.onnx")
    # Xilinx Spartan-7 Spec
    spec = svimpl.FPGASpec(120, 600_000, 2_700_000, 100_000)
    fpga_module = parsemodel.gen_chain_sv_top(mdl, dim, spec)
    fpga_module.alloc_regs()
    fpga_module.alloc_bram("")
    fpga_module.tlname = tlname
    sv = fpga_module.make_sv()

    return mdl, sv, fpga_module

In [17]:
import cocotb
import numpy as np
import onnx
from cocotb.runner import get_runner

In [18]:
from cocotb.utils import get_sim_time as gst
from onnx import TensorProto
from onnx import helper as onnxhelp
from bespoketb.bespoke import *
from bespoketb.boards import EpsScoreboard
from bespoketb.ctb_util import *

In [19]:
test = """
class DemoTester(BspkModuleTester):
    def __init__(self, dut, modelpath, in_dim, debug, **kwargs):
        board = EpsScoreboard(0.01, debug, dut)
        super().__init__(dut, modelpath, in_dim, board, False, debug, **kwargs)

    def _gen_indata(self):
        return 0.2*np.random.randn(self.in_dim).astype(dtype=np.float32)
CYCLES = 10
@cocotb.test()
async def demo_test(dut):
    #cocotb test for vwbmacc
    tlen = int(os.getenv("TLEN"))
    mdl_dim = int(os.getenv("MDIM"))
    mdl_path = os.getenv("MPATH")
    debug = os.getenv("DEBUG") == "True"
    if debug:
        print("DEBUGGING", os.environ)
    test = DemoTester(dut, mdl_path, mdl_dim, debug)
    await test.startup()

    await test.run_io_test(tlen)

    await ClockCycles(dut.clk_in, tlen * CYCLES)
    dut._log.info(
        f"Processed {test.inm.stats.received_transactions} transactions in and {test.outm.stats.received_transactions} out"
    )
    assert (
        test.inm.stats.received_transactions == test.outm.stats.received_transactions
    ), f"Transaction Count doesn't match! :/"
    dut._log.info(
        f"maximum float err between recieved and expected: {test.scoreboard.maxerr}"
    )
"""

In [20]:
import inspect

In [21]:
imports = """
import os
import cocotb
from bespoketb.bespoke import *
from bespoketb.boards import EpsScoreboard
from bespoketb.ctb_util import *
"""
with open("demo_test.py", 'w') as f:
    f.write(imports)
    f.write(test)

In [22]:
import os
import sys
from pathlib import Path
def test_runner():
    """Simulate the counter using the Python runner."""
    sim = os.getenv("SIM", "icarus")
    proj_path = Path(os.path.abspath(".."))
    print(proj_path)
    sys.path.append(str(proj_path / "sim" / "model"))

    sys.path.append(str(proj_path / "sim"))

    runner = get_runner(sim)
    runner.build(
        hdl_toplevel="demo_dummy_tl",
        always=True,
        build_args=["-Wall"],
        timescale=("1ns", "1ps"),
        waves=True,
        verilog_sources = ["../hdl/interface/v_fifo.sv",
                           "../hdl/mlops/vwb_gemm.sv",
                           "../hdl/mlops/mac1d.sv",
                           "../hdl/mlops/addertree.sv",
                           "../hdl/mlops/v_leakyrelu.sv",
                           "../hdl/dummymodels/demo_dummy_tl.sv",
                           "../hdl/interface/xilinx_single_port_ram_read_first.v" ],
        log_file="dump.log",
        verbose=True
        )

    # sys.path.append(str(Path.absolute("../sim")))
    runner.test(
        test_module="demo_test",
        hdl_toplevel="demo_dummy_tl",
        extra_env = {"MDIM":"10", "MPATH":"data/demo_model.onnx", "TLEN":"25", "DEBUG":"True"},
        verbose=True,
        waves=True
    )


In [24]:
tlname = "demo_dummy_tl"
uut_mdl, sv, impl = make_testing_tl(10, tlname)
with open(f"../hdl/dummymodels/{tlname}.sv", "w") as f:
        f.write(sv)
test_runner()

WRITING BRAM WITH SHAPE (20, 10) 2 TO data\_lin1_Gemm_weight.mem CHUNKS 10
WRITING BRAM WITH SHAPE (20,) 1 TO data\_lin1_Gemm_bias.mem CHUNKS 1
WRITING BRAM WITH SHAPE (40, 20) 2 TO data\_lin2_Gemm_weight.mem CHUNKS 20
WRITING BRAM WITH SHAPE (40,) 1 TO data\_lin2_Gemm_bias.mem CHUNKS 1
WRITING BRAM WITH SHAPE (3, 40) 2 TO data\_lin3_Gemm_weight.mem CHUNKS 40
WRITING BRAM WITH SHAPE (3,) 1 TO data\_lin3_Gemm_bias.mem CHUNKS 1
c:\Users\TheoA\Documents\MIT\SENIORSLIDE\6.S965 Digital Systems Lab II\Final Proj\audio-rc-rtl
INFO: Running command iverilog -o 'C:\Users\TheoA\Documents\MIT\SENIORSLIDE\6.S965 Digital Systems Lab II\Final Proj\audio-rc-rtl\sim\sim_build\sim.vvp' -D COCOTB_SIM=1 -s demo_dummy_tl -g2012 -Wall -s cocotb_iverilog_dump -f 'C:\Users\TheoA\Documents\MIT\SENIORSLIDE\6.S965 Digital Systems Lab II\Final Proj\audio-rc-rtl\sim\sim_build\cmds.f' 'C:\Users\TheoA\Documents\MIT\SENIORSLIDE\6.S965 Digital Systems Lab II\Final Proj\audio-rc-rtl\hdl\interface\v_fifo.sv' 'C:\Users\

In [25]:
with open(os.path.abspath("./sim_build/results.xml"), 'r') as f:
    for l in f.readlines():
        print(l)

<testsuites name="results">

  <testsuite name="all" package="all">

    <property name="random_seed" value="1733975588" />

    <testcase name="demo_test" classname="demo_test" file="c:\Users\TheoA\Documents\MIT\SENIORSLIDE\6.S965 Digital Systems Lab II\Final Proj\audio-rc-rtl\sim\demo_test.py" lineno="16" time="0.023839712142944336" sim_time_ns="0.001" ratio_time="0.0419468152133692">

      <failure message="Test failed with RANDOM_SEED=1733975588" />

    </testcase>

  </testsuite>

</testsuites>



In [27]:
impl

(-1, 10)
('vwb_gemm_0', (10, 20))
fifo
('v_leakyrelu_0', (20, 20))
fifo
('vwb_gemm_1', (20, 40))
fifo
('v_leakyrelu_1', (40, 40))
fifo
('vwb_gemm_2', (40, 3))
(3, -1)